In [1]:
!pip install transformers
!pip install torchvision
!pip install SentencePiece 
import os
import math

import torch
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 1.9MB 5.8MB/s 
     |████████████████████████████████| 890kB 26.9MB/s 
     |████████████████████████████████| 3.2MB 18.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=2b64a584355a2bea03457d3f1d565a26a6a02363ec637539f13900b0922b25cc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 5.6MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Tokenzation and Attention

In [3]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

In [4]:
def tokenize_inputs(text_list, tokenizer, num_embeddings=512):
    """
    Tokenizes the input text input into ids. Appends the appropriate special
    characters to the end of the text to denote end of sentence. Truncate or pad
    the appropriate sequence length.
    """
    # tokenize the text, then truncate sequence to the desired length minus 2 for
    # the 2 special characters
    tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:num_embeddings-2], text_list))
    # convert tokenized text into numeric ids for the appropriate LM
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    # append special token "<s>" and </s> to end of sentence
    input_ids = [tokenizer.build_inputs_with_special_tokens(x) for x in input_ids]
    # pad sequences
    input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post")
    return input_ids

def create_attn_masks(input_ids):
    """
    Create attention masks to tell model whether attention should be applied to
    the input id tokens. Do not want to perform attention on padding tokens.
    """
    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return attention_masks

## Load fintuned model 

In [5]:
def load_model(save_path):
    """
    Load the model from the path directory provided
    """
    checkpoint = torch.load(save_path)
    model_state_dict = checkpoint['state_dict']
    model = XLNetForMultiLabelSequenceClassification(num_labels=model_state_dict["classifier.weight"].size()[0])
    model.load_state_dict(model_state_dict)

    epochs = checkpoint["epochs"]
    lowest_eval_loss = checkpoint["lowest_eval_loss"]
    train_loss_hist = checkpoint["train_loss_hist"]
    valid_loss_hist = checkpoint["valid_loss_hist"]
  
    return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist

In [6]:
torch.cuda.empty_cache()

In [1]:
config = XLNetConfig()
        
class XLNetForMultiLabelSequenceClassification(torch.nn.Module):
    def __init__(self, num_labels=2):
        super(XLNetForMultiLabelSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')
        self.classifier = torch.nn.Linear(768, num_labels)

        torch.nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels=None):
    # last hidden layer
        last_hidden_state = self.xlnet(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)
    # pool the outputs into a mean vector
        mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
        logits = self.classifier(mean_last_hidden_state)
        
        if labels is not None:
            loss_fct = BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels),\
                      labels.view(-1, self.num_labels))
            return loss
        else:
            return logits
    
    def freeze_xlnet_decoder(self):
        """
        Freeze XLNet weight parameters. They will not be updated during training.
        """
        for param in self.xlnet.parameters():
            param.requires_grad = False
    
    def unfreeze_xlnet_decoder(self):
        """
        Unfreeze XLNet weight parameters. They will be updated during training.
        """
        for param in self.xlnet.parameters():
            param.requires_grad = True
    
    def pool_hidden_state(self, last_hidden_state):
        """
        Pool the output vectors into a single mean vector 
        """
        last_hidden_state = last_hidden_state[0]
        mean_last_hidden_state = torch.mean(last_hidden_state, 1)
        return mean_last_hidden_state
    
# model = XLNetForMultiLabelSequenceClassification(num_labels=len(Y_train[0]))
# model = torch.nn.DataParallel(model)
# model.cuda()

NameError: name 'XLNetConfig' is not defined

In [8]:
cwd = os.getcwd()
model_save_path = output_model_file = os.path.join(cwd, "/content/drive/My Drive/xlnet_toxic.bin")
model, start_epoch, lowest_eval_loss, train_loss_hist, valid_loss_hist = load_model(model_save_path)

In [9]:
def generate_predictions(model, df, num_labels, device="cpu", batch_size=32):
    num_iter = math.ceil(df.shape[0]/batch_size)
  
    pred_probs = np.array([]).reshape(0, num_labels)
  
    model.to(device)
    model.eval()
  
    for i in range(num_iter):
        df_subset = df.iloc[i*batch_size:(i+1)*batch_size,:]
        X = df_subset["features"].values.tolist()
        masks = df_subset["masks"].values.tolist()
        X = torch.tensor(X)
        masks = torch.tensor(masks, dtype=torch.long)
        X = X.to(device)
        masks = masks.to(device)
        with torch.no_grad():
            logits = model(input_ids=X, attention_mask=masks)
            logits = logits.sigmoid().detach().cpu().numpy()
            pred_probs = np.vstack([pred_probs, logits])
  
    return pred_probs

## Load Meeting Transcript

In [10]:
import pandas as pd
df=pd.read_csv('/content/sample_data/transcript.txt',names=["sent"],sep='/:')
df.to_csv('someFileName.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [11]:
df_list=df.values.tolist()
df.head()

,sent
0,"p1 : hi guys, good morning."
1,p2 : hi boss.
2,p1 : how are you all?
3,p2 : good boss.
4,p2 : fine boss.


In [12]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
new_list=[]
for i in df_list:
    j=str(i).strip("[]'")
    indx = j.find(":")#position of 'I'
    intro = j[indx+1:]
    new_list.append(intro)
# re.sub(r'[a-zA-Z]+[:]', 'I', stri)
# flat_list = [item for sublist in new_list for item in sublist]
new_list

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[' hi guys, good morning.',
 ' hi boss.',
 ' how are you all?',
 ' good boss.',
 ' fine boss.',
 ' what is status of hexa application development?',
 ' we have completed four modules and started the fifth module.',
 ' and quality and testing also happening in prallel.',
 ' okay guys, but we have to complete all the phases of development by end of this month.',
 ' and we have to gic demo to user right, when will be that ?',
 ' we have to give demo to the user on first week of next month, so be ready.',
 ' please start the frontend work and complete it by end of the day.',
 ' okay',
 ' okay keep me updated. take care',
 ' okay we will be, thank you.']

In [13]:
import numpy as np
test_np=np.array(new_list)
test_np

array([' hi guys, good morning.', ' hi boss.', ' how are you all?',
       ' good boss.', ' fine boss.',
       ' what is status of hexa application development?',
       ' we have completed four modules and started the fifth module.',
       ' and quality and testing also happening in prallel.',
       ' okay guys, but we have to complete all the phases of development by end of this month.',
       ' and we have to gic demo to user right, when will be that ?',
       ' we have to give demo to the user on first week of next month, so be ready.',
       ' please start the frontend work and complete it by end of the day.',
       ' okay', ' okay keep me updated. take care',
       ' okay we will be, thank you.'], dtype='<U87')

In [14]:
# create input id tokens
test_np_input_ids = tokenize_inputs(test_np, tokenizer, num_embeddings=250)
test_np_input_ids

array([[  17, 2582, 2500, ...,    0,    0,    0],
       [  17, 2582, 5643, ...,    0,    0,    0],
       [ 160,   41,   44, ...,    0,    0,    0],
       ...,
       [4968,    4,    3, ...,    0,    0,    0],
       [4968,  435,  110, ...,    0,    0,    0],
       [4968,   80,   53, ...,    0,    0,    0]])

In [15]:
# create attention masks
test_np_attention_masks = create_attn_masks(test_np_input_ids)
test_np_attention_masks

[[1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [16]:
import pandas as pd
dataset_unseen = pd.DataFrame()
dataset_unseen['Sent'] = test_np.tolist()
dataset_unseen.shape

(15, 1)

In [17]:
dataset_unseen["features"] = test_np_input_ids.tolist()
dataset_unseen["masks"] = test_np_attention_masks

In [18]:
dataset_unseen1=dataset_unseen[["Sent","features","masks"]]
dataset_unseen1.head()

,Sent,features,masks
0,"hi guys, good morning.","[17, 2582, 2500, 19, 195, 907, 9, 4, 3, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,hi boss.,"[17, 2582, 5643, 9, 4, 3, 0, 0, 0, 0, 0, 0, 0,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
2,how are you all?,"[160, 41, 44, 71, 82, 4, 3, 0, 0, 0, 0, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ..."
3,good boss.,"[195, 5643, 9, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,fine boss.,"[1592, 5643, 9, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


In [19]:
label_cols = ["label_ami_da_1","label_ami_da_11","label_ami_da_12","label_ami_da_13","label_ami_da_14","label_ami_da_15","label_ami_da_16","label_ami_da_2","label_ami_da_3","label_ami_da_4","label_ami_da_5","label_ami_da_6","label_ami_da_7","label_ami_da_8","label_ami_da_9"]
num_labels = len(label_cols)
pred_probs = generate_predictions(model, dataset_unseen1, num_labels, device="cuda", batch_size=32)
pred_probs

array([[3.30025912e-04, 2.20747697e-05, 7.27576917e-05, 3.53462747e-05,
        9.84468639e-01, 7.31008884e-04, 6.63802610e-04, 5.34112449e-04,
        1.55040040e-03, 9.79823875e-04, 4.44534853e-05, 2.19158290e-04,
        1.00668396e-04, 3.60789163e-05, 1.91076528e-02],
       [6.80221303e-04, 2.29944235e-05, 1.57297909e-04, 4.13627640e-05,
        9.76672173e-01, 2.08186405e-03, 1.83337566e-03, 8.18511297e-04,
        1.44487678e-03, 1.32867135e-03, 7.44875724e-05, 2.63738242e-04,
        1.08232911e-04, 4.09424174e-05, 2.99324784e-02],
       [2.63468456e-03, 4.05029386e-01, 2.65751742e-02, 1.42798126e-02,
        4.06741276e-02, 3.69253382e-03, 6.96747974e-02, 7.29731796e-03,
        4.08373289e-02, 1.89808514e-02, 3.34148109e-01, 9.42639913e-03,
        3.06589343e-03, 1.37632759e-02, 2.02325657e-02],
       [2.23900937e-02, 5.06621727e-04, 1.66762364e-03, 1.11159949e-04,
        4.17757779e-01, 7.19223544e-03, 1.21425306e-02, 2.29987944e-03,
        1.92048284e-03, 7.41808861e-0

In [20]:
# label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
label_cols = ["label_ami_da_1","label_ami_da_11","label_ami_da_12","label_ami_da_13","label_ami_da_14","label_ami_da_15","label_ami_da_16","label_ami_da_2","label_ami_da_3","label_ami_da_4","label_ami_da_5","label_ami_da_6","label_ami_da_7","label_ami_da_8","label_ami_da_9"]
dataset_unseen1["label_ami_da_1"] = pred_probs[:,0]
dataset_unseen1["label_ami_da_11"] = pred_probs[:,1]
dataset_unseen1["label_ami_da_12"] = pred_probs[:,2]
dataset_unseen1["label_ami_da_13"] = pred_probs[:,3]
dataset_unseen1["label_ami_da_14"] = pred_probs[:,4]
dataset_unseen1["label_ami_da_15"] = pred_probs[:,5]
dataset_unseen1["label_ami_da_16"] = pred_probs[:,6]
dataset_unseen1["label_ami_da_2"] = pred_probs[:,7]
dataset_unseen1["label_ami_da_3"] = pred_probs[:,8]
dataset_unseen1["label_ami_da_4"] = pred_probs[:,9]
dataset_unseen1["label_ami_da_5"] = pred_probs[:,10]
dataset_unseen1["label_ami_da_6"] = pred_probs[:,11]
dataset_unseen1["label_ami_da_7"] = pred_probs[:,12]
dataset_unseen1["label_ami_da_8"] = pred_probs[:,13]
dataset_unseen1["label_ami_da_9"] = pred_probs[:,14]

In [21]:
dataset_unseen1['HighScore'] = dataset_unseen1.max(axis=1)

In [22]:
dataset_unseen1

,Sent,features,masks,label_ami_da_1,label_ami_da_11,label_ami_da_12,label_ami_da_13,label_ami_da_14,label_ami_da_15,label_ami_da_16,label_ami_da_2,label_ami_da_3,label_ami_da_4,label_ami_da_5,label_ami_da_6,label_ami_da_7,label_ami_da_8,label_ami_da_9,HighScore
0,"hi guys, good morning.","[17, 2582, 2500, 19, 195, 907, 9, 4, 3, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.000330,0.000022,0.000073,0.000035,0.984469,0.000731,0.000664,0.000534,0.001550,0.000980,0.000044,0.000219,0.000101,0.000036,0.019108,0.984469
1,hi boss.,"[17, 2582, 5643, 9, 4, 3, 0, 0, 0, 0, 0, 0, 0,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",0.000680,0.000023,0.000157,0.000041,0.976672,0.002082,0.001833,0.000819,0.001445,0.001329,0.000074,0.000264,0.000108,0.000041,0.029932,0.976672
2,how are you all?,"[160, 41, 44, 71, 82, 4, 3, 0, 0, 0, 0, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",0.002635,0.405029,0.026575,0.014280,0.040674,0.003693,0.069675,0.007297,0.040837,0.018981,0.334148,0.009426,0.003066,0.013763,0.020233,0.405029
3,good boss.,"[195, 5643, 9, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.022390,0.000507,0.001668,0.000111,0.417758,0.007192,0.012143,0.002300,0.001920,0.007418,0.000333,0.001576,0.000862,0.000185,0.565789,0.565789
4,fine boss.,"[1592, 5643, 9, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.035472,0.001473,0.005395,0.000272,0.439377,0.010509,0.018414,0.003298,0.002422,0.021883,0.001054,0.003465,0.001589,0.000351,0.538148,0.538148
5,what is status of hexa application development?,"[113, 27, 1855, 20, 17, 27811, 1479, 503, 82, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.000597,0.033567,0.003509,0.000741,0.002574,0.000398,0.007007,0.000284,0.005703,0.006751,0.963006,0.001724,0.000833,0.010319,0.005999,0.963006
6,we have completed four modules and started th...,"[80, 47, 1570, 237, 12772, 21, 505, 18, 2287, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.001441,0.000928,0.005130,0.000058,0.001911,0.000725,0.010470,0.000029,0.000163,0.971105,0.018924,0.000399,0.000862,0.000056,0.013108,0.971105
7,and quality and testing also happening in pra...,"[21, 882, 21, 2677, 77, 4456, 25, 17, 7627, 21...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.000650,0.002625,0.001680,0.000143,0.003303,0.001148,0.006081,0.000505,0.007878,0.931980,0.012971,0.038130,0.014537,0.001239,0.013788,0.931980
8,"okay guys, but we have to complete all the ph...","[4968, 2500, 19, 57, 80, 47, 22, 1009, 71, 18,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.001702,0.000941,0.004577,0.000079,0.034733,0.000842,0.006611,0.000072,0.000591,0.664918,0.008934,0.203924,0.008842,0.000909,0.106795,0.664918
9,"and we have to gic demo to user right, when w...","[21, 80, 47, 22, 17, 14614, 9578, 22, 1930, 20...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.000528,0.091930,0.005688,0.000486,0.006403,0.000377,0.007113,0.000659,0.009351,0.074043,0.619538,0.198392,0.006579,0.211643,0.033978,0.619538


In [23]:
Dict_label = {"label_ami_da_1":"Backchannel","label_ami_da_11":"Elicit-Assessment","label_ami_da_12":"Comment-About-Understanding","label_ami_da_13":"Elicit-Comment-Understanding","label_ami_da_14":"Be-Positive","label_ami_da_15":"Be-Negative","label_ami_da_16":"Other","label_ami_da_2":"Stall","label_ami_da_3":"Fragment","label_ami_da_4":"Inform","label_ami_da_5":"Elicit-Inform","label_ami_da_6":"Suggest","label_ami_da_7":"Offer","label_ami_da_8":"Elicit-Offer-Or-Suggestion","label_ami_da_9":"Assess"}
new_lst=[]
new_lst_label_name=[]
d=dataset_unseen1.columns[3:-1]
l=len(dataset_unseen1)
for i in range(l):
    for col in d:
        if dataset_unseen1[col][i] == dataset_unseen1["HighScore"][i]:
            new_lst.append(col)
            for k, v in Dict_label.items():
                if k == col:
                    new_lst_label_name.append(v)


      

In [24]:
dataset_unseen1["Label"]=new_lst
dataset_unseen1["Label_name"]=new_lst_label_name
dataset_unseen1[["Sent","HighScore","Label","Label_name"]].to_csv("output_action_itmes")


## Trying to implement Abstractive Dialogue Summarization 

In [40]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch


model_name = 'google/pegasus-xsum'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
# batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding='longest', return_tensors="pt").to(torch_device)
# translated = model.generate(**batch)
# tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
# assert tgt_text[0] == "California's largest electricity provider has turned off power to hundreds of thousands of customers."

NameError: ignored

In [41]:
src_text = [
    """ we have completed four modules and started the fifth module and quality and testing also happening in prallel.
     we have to give demo to the user on first week of next month, so be ready. please start the frontend work and complete it by end of the day. """
]
batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding='longest', return_tensors="pt").to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
tgt_text


['we are working on a new website and want to start work on it as soon as possible.']